In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Attention, Bidirectional, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [19]:
# Load the IMDb dataset
(train_texts, train_labels), (test_texts, test_labels) = tf.keras.datasets.imdb.load_data()


In [20]:
word_index = tf.keras.datasets.imdb.get_word_index()

# Reverse the word index to map indices back to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Decode the review texts
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# Apply the decoding to the train and test texts
train_texts = [decode_review(text) for text in train_texts]
test_texts = [decode_review(text) for text in test_texts]

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=256)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=256)


In [21]:
# Define the model
vocab_size = 25000
embedding_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout_rate = 0.5



In [ ]:
model = tf.keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=256),
    Bidirectional(LSTM(hidden_dim, return_sequences=True)),
    Attention()([LSTM(hidden_dim, return_sequences=True),
                 LSTM(hidden_dim, return_sequences=True)]),
    Dense(output_dim, activation='sigmoid')
])

In [ ]:
# Compile the model
optimizer = Adam()
loss_function = BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Training the model
batch_size = 64
num_epochs = 5

model.fit(train_padded, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(test_padded, test_labels))